In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

num_classes = 4
num_features = 2
random_seed = 42

xBlob, yBlob = make_blobs(n_samples=1000, centers=num_classes, n_features=num_features, random_state=random_seed, cluster_std=1.5)
xBlob = torch.from_numpy(xBlob).type(torch.float32)
yBlob = torch.from_numpy(yBlob).type(torch.long)

X_train, X_test, y_train, y_test = train_test_split(xBlob, yBlob, test_size=0.2, random_state=random_seed)

plt.figure(figsize=(10,7))
plt.scatter(xBlob[:,0], xBlob[:,1], c=yBlob, cmap='rainbow')
plt.show()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
from torch import nn as nn
class multiclassmodel(nn.Module):
    def __init__(self, input_features, output_features, hidden_units=8):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_features)
        )

    def forward(self, x):
        return self.linear(x)

model = multiclassmodel(input_features=2,output_features=4)
model.to(device)
model

In [ ]:
lossFn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [ ]:
def acc(true, pred):
  count = 0.0
  for i in range(pred.size(dim=0)):
      if pred[i] == true[i]:
          count+=1
  return count/pred.size(dim=0)


In [ ]:
epochs = 100
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

for epoch in range(epochs):
    model.train()
    logits = model(X_train)
    ypred = torch.softmax(logits, dim=1).argmax(dim=1)
    loss = lossFn(logits, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    with torch.inference_mode():
      tlogits = model(X_test)
      tpred = torch.softmax(tlogits, dim=1).argmax(dim=1)
      Tloss = lossFn(tlogits, y_test)
      tacc = acc(y_test, tpred)
      if epoch % 10 == 0:
        print(f"{epoch} ____ {loss} ____ {Tloss} ,,,, {tacc}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

def plot_decision_boundary(model, X, y, device, title='Decision Boundary'):
    # Set the limits for the grid
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))

    # Flatten the grid to pass through the model
    grid = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32).to(device)

    # Put the model in evaluation mode and get predictions
    model.eval()
    with torch.no_grad():
        logits = model(grid)
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1).cpu().numpy()

    # Reshape the predictions to match the grid shape
    Z = preds.reshape(xx.shape)

    # Create the plot
    plt.figure(figsize=(10, 6))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap='viridis', edgecolor='k')
    plt.title(title)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.colorbar(label='Class')
    plt.show()

# Example usage:
# Assuming X_test and y_test are already defined and moved to CPU for plotting
X_test_np = X_train.cpu().numpy()
y_test_np = y_train.cpu().numpy()

# Call the function with your model and test data
plot_decision_boundary(model, X_test_np, y_test_np, device, title='Decision Boundary on Test Data')


In [ ]:
X_test_np = X_test.cpu().numpy()
y_test_np = y_test.cpu().numpy()

plot_decision_boundary(model, X_test_np, y_test_np, device, title='Decision Boundary on Test Data')